In [1]:
from bs4 import BeautifulSoup
import json
import urllib
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
import time
import json


Scrapping from GlassDoor

In [9]:
# Setup Firefox options
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("--headless")  # Run in headless mode (without a GUI)
firefox_options.add_argument("--no-sandbox")
firefox_options.add_argument("--disable-dev-shm-usage")

# Start the WebDriver
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

# URL of the job listings page
# url = "https://www.glassdoor.co.in/Job/jobs.htm?suggestCount=0&suggestChosen=true&clickSource=searchBtn&typedKeyword=it&sc.keyword=software%20developer%20intern&locT=C&locId=4477468&jobType=fulltime&fromAge=1&radius=6&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0"  # Replace with the actual URL
# url = "https://www.glassdoor.co.in/Job/pune-india-data-analyst-jobs-SRCH_IL.0,10_IC2856202_KO11,23.htm"  # Replace with the actual URL
url = "https://www.glassdoor.co.in/Job/india-web-developer-jobs-SRCH_IL.0,5_IN115_KO6,19.htm?locId=115&locT=N&sc.keyword=Web%20Developer"  # Replace with the actual URL
driver.get(url)

# Wait for the page to load
time.sleep(10)

# Locate the job listing elements
job_listings = driver.find_elements(By.CLASS_NAME, "JobsList_jobListItem__wjTHv")
job_data=[]
count = 0
for job in job_listings:
    
    if count > 50:
        break
    try:
        # Extract job title
        title_element = job.find_element(By.CLASS_NAME, "JobCard_jobTitle___7I6y")
        job_title = title_element.text
        job_link = title_element.get_attribute("href")  # Get the link to the job
        
        # Extract employer name
        employer_element = job.find_element(By.CLASS_NAME, "EmployerProfile_compactEmployerName__LE242")
        employer_name = employer_element.text
        
        # Extract location
        location_element = job.find_element(By.CLASS_NAME, "JobCard_location__rCz3x")
        job_location = location_element.text
        
        # Extract salary
        salary_element = job.find_element(By.CLASS_NAME, "JobCard_salaryEstimate__arV5J")
        job_salary = salary_element.text
        
        job_data.append({
            "Position": job_title,
            "Company": employer_name,
            "Location": job_location,
            "Salary": job_salary,
            "url": job_link
        })
        count += 1
        # print("Salary:",job_salary)
    except Exception as e:
        print("Error extracting information:", e)

with open('job_listings.json', 'w') as json_file:
    json.dump(job_data, json_file, indent=4)

print("Job listings saved to job_listings.json")
print(len(job_listings))
# Close the WebDriver
driver.quit()


Error extracting information: Message: Unable to locate element: .JobCard_salaryEstimate__arV5J; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Error extracting information: Message: Unable to locate element: .JobCard_salaryEstimate__arV5J; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver

In [11]:
import re

def summarize_job_description(job_text):
    # Extracting responsibilities using keywords commonly associated with duties
    responsibilities = re.findall(r"(?:Design|Integrate|Identify|Help|Translate|Ensure|Build|Work|Debug|Customise|Demonstrate|Carry)\b.*", job_text)
    
    # Extracting tech stack keywords
    tech_stack_match = re.search(r"Proficient in \{\{(.+?)\}\}", job_text)
    tech_stack = tech_stack_match.group(1) if tech_stack_match else "Not mentioned"
    
    # Requirements and skills
    requirements = re.findall(r"(?:Proficient|Strong understanding|Skill|Familiar|Experience)\b.*", job_text)
    
    # Joining extracted info into a single paragraph
    responsibilities_str = " ".join(responsibilities)
    requirements_str = " ".join(requirements)
    
    # Final summary
    summary = (f"{responsibilities_str} The candidate should have proficiency in {tech_stack}. "
               f"Additional skills include {requirements_str}.")
    
    return summary

In [10]:
# URL of the job listings page
job_listings = []
with open('job_listings.json') as f:
    job_listings = json.load(f)

for job in job_listings:
    url = job['url']   # Replace with the actual URL
    driver.get(url)
    time.sleep(10)
    
    all_data = ""  # Initialize all_data to an empty string

    try:
        # Find the job details container
        job_details_container = driver.find_element(By.CLASS_NAME, "JobDetails_jobDetailsContainer__y9P3L")

        # Find and click the "Show more" button
        show_more_button = job_details_container.find_element(By.XPATH, ".//*[contains(text(), 'Show more')]")
        show_more_button.click()

        # Wait for a moment to ensure the new details are loaded
        time.sleep(5)  # You might want to adjust this based on loading times or use WebDriverWait

        # Get all the text under that container after clicking "Show more"
        all_data = job_details_container.text

    except NoSuchElementException:
        print(f"Element not found for URL: {url}. Setting job description to empty.")
        # all_data remains an empty string

    # Summarize the all_data
    # summary = summarize_job_description(all_data)
    job['Job_Description'] = all_data
    
    print(job['Company'])

# Save the updated job listings with job descriptions
jd_list = []
with open('job_listings_with_jd.json') as f:
    jd_list = json.load(f)

jd_list.append(job_listings)

with open('job_listings_with_jd.json', 'w') as json_file:
    json.dump(jd_list, json_file, indent=4)

# Close the WebDriver
driver.quit()


Confidential
Zanskar Tech Pvt Ltd
Frnsup Technologies Pvt. Ltd.
Parakh Online
Giraf
Appening Infotech Pvt. Ltd.
Edrevel
Decision Management Solutions
Comtrek Digital Pvt Ltd
Giraf
Pristine Technologies
Prachas Technologies
InScience Healthcare Consulting Pvt. Ltd.
SUPERCODER INC
Yashaa Digital
Uniwise Technology Labs Pvt Ltd
Finpoint
Emmsons Infotech
Zenith Future LLP
Collablearn
OneModo Technologies Pvt Ltd
EPIA
Frugal Testing


In [47]:
def scrape_job_description(url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load completely
    job_description = {}

    try:
        # Scrape Key Responsibilities
        responsibilities_title = driver.find_element(By.XPATH, "//*[contains(., 'Responsibilit')]")
        responsibilities_list = responsibilities_title.find_element(By.XPATH, "following-sibling::ul").text
        job_description['Key Responsibilities'] = responsibilities_list.split("\n")
        
        # Scrape Requirements
        requirements_title = driver.find_element(By.XPATH, "//*[contains(., 'Requirement')]")
        requirements_list = requirements_title.find_element(By.XPATH, "following-sibling::ul").text
        job_description['Requirements'] = requirements_list.split("\n")

    except Exception as e:
        print(f"Could not scrape job description from {url}: {e}")
        job_description['Key Responsibilities'] = "Not found"
        job_description['Requirements'] = "Not found"
    
    print(job_description)
    return job_description


with open('demo.json') as f:
    job_listings = json.load(f)

# Loop through each job in the JSON and scrape the job description
for job in job_listings:
    job_url = job.get("Job Link")
    job_details = scrape_job_description(job_url)
    job["Job Description"] = job_details
    print(f"Scraped description for {job['Job Title']}")

# Save the updated job listings with descriptions to a new JSON file
with open('demo2.json', 'w') as f:
    json.dump(job_listings, f, indent=4)

# Close the browser
driver.quit()

Could not scrape job description from https://www.glassdoor.co.in/job-listing/andoid-app-developer-intern-constems-ai-JV_IC4477468_KO0,27_KE28,39.htm?jl=1009485312339: Message: Unable to locate element: following-sibling::ul; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

{'Key Responsibilities': 'Not found', 'Requirements': 'Not found'}
Scraped description for Andoid App Developer Intern
Could not scrape job description from https://www.glassdoor.co.in/job-listing/backend-developer-intern-international-youth-edu-skills-foundation-JV_IC4477468_KO0,24_KE25,66.htm?jl=1009377074499: Message: Unabl

Scrapping from SimplyHired

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import time
import json

# Setup Firefox options
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("--headless")  # Run in headless mode (without a GUI)
firefox_options.add_argument("--no-sandbox")
firefox_options.add_argument("--disable-dev-shm-usage")

# Start the WebDriver
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

# URL of the job listings page
url = "https://www.simplyhired.com/search?q=ml+engineer&l="  # Replace with the actual URL
driver.get(url)

# Wait for the page to load
time.sleep(10)

# Locate the job listing elements (updated selector)
job_listings = driver.find_elements(By.CSS_SELECTOR, "[data-testid='searchSerpJob']")
job_data = []
count = 0

for job in job_listings:
    if count > 50:
        break
    try:
        # Extract job title
        title_element = job.find_element(By.CSS_SELECTOR, "[data-testid='searchSerpJobTitle'] a")
        job_title = title_element.text
        job_link = title_element.get_attribute("href")
        
        # Extract employer name
        employer_element = job.find_element(By.CSS_SELECTOR, "[data-testid='companyName']")
        employer_name = employer_element.text
        
        # Extract location
        location_element = job.find_element(By.CSS_SELECTOR, "[data-testid='searchSerpJobLocation']")
        job_location = location_element.text
        
        # Extract salary (if available)
        try:
            salary_element = job.find_element(By.CSS_SELECTOR, "[data-testid='searchSerpJobSalaryConfirmed']")
            job_salary = salary_element.text
        except:
            job_salary = "Not specified"
        
        # Maintain the original JSON structure
        job_data.append({
            "Position": job_title,
            "Company": employer_name,
            "Location": job_location,
            "Salary": job_salary,
            "url": job_link
        })
        count += 1
        
    except Exception as e:
        print("Error extracting information:", e)

# Save the data to JSON file
with open('job_listings_simplyhired.json', 'w') as json_file:
    json.dump(job_data, json_file, indent=4)

print("Job listings saved to job_listings.json")
print(len(job_listings))

# Close the WebDriver
driver.quit()

Job listings saved to job_listings.json
20


In [5]:
# Setup Firefox options
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("--headless")  # Run in headless mode (without a GUI)
firefox_options.add_argument("--no-sandbox")
firefox_options.add_argument("--disable-dev-shm-usage")

# Start the WebDriver
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

# URL of the job listings page
job_listings = []
with open('job_listings_simplyhired.json') as f:
    job_listings = json.load(f)

for job in job_listings:
    url = job['url']   # Replace with the actual URL
    driver.get(url)
    time.sleep(10)
    
    all_data = ""  # Initialize all_data to an empty string

    try:
        # Find the job details container
        job_details_container = driver.find_element(By.CLASS_NAME, "css-k008qs")
        time.sleep(5)

        all_data = job_details_container.text

    except NoSuchElementException:
        print(f"Element not found for URL: {url}. Setting job description to empty.")
        # all_data remains an empty string

    # Summarize the all_data
    # summary = summarize_job_description(all_data)
    job['Job_Description'] = all_data
    
    print(job['Company'])

# Save the updated job listings with job descriptions
# jd_list = []
# with open('job_listings_with_jd_simplyhired.json') as f:
#     jd_list = json.load(f)

# jd_list.append(job_listings)

with open('job_listings_with_jd_simplyhired.json', 'w') as json_file:
    json.dump(job_listings, json_file, indent=4)

# Close the WebDriver
driver.quit()


Global Path Consulting Services LLC
Netflix
Deep 6 AI
Ccube
a16z
General Technical Services, LLC
Intone Networks
Children's National Hospital
Xero
General Motors
Moonwalk Biosciences, Inc.
Apple
Flagship Pioneering, Inc.
Unlimit Ventures
DuploCloud
Pioneer Square Labs
G2M Talent
Character.AI
1st Edge, LLC
Serve Robotics


In [6]:
import pandas as pd

# Load the JSON file
json_file_path = 'job_listings_with_jd_simplyhired.json'
data = pd.read_json(json_file_path,orient='records')

# Convert to CSV
csv_file_path = 'data_simplyhired.csv'
data.to_csv(csv_file_path, index=False)

print(f"Converted {json_file_path} to {csv_file_path}")


Converted job_listings_with_jd_simplyhired.json to data_simplyhired.csv


In [9]:
df1=pd.read_csv("data.csv")
df2=pd.read_csv("data_simplyhired.csv")
# df1.head(30)
print(df1)
df2

                       Position                                    Company  \
0      ReactJS Developer intern                                Constems-AI   
1      ReactJS Developer Intern                        Socialveins Pvt Ltd   
2      Backend Developer Intern  INTERNATIONAL YOUTH EDU-SKILLS FOUNDATION   
3   Andoid App Developer Intern                                Constems-AI   
4         Java Developer Intern                              Parakh Online   
..                          ...                                        ...   
59             We Web Developer                          Zenith Future LLP   
60         Web Developer Intern                                Collablearn   
61     Junior Laravel Developer               OneModo Technologies Pvt Ltd   
62         Web Developer Remote                                       EPIA   
63      Sr. Front End Developer                             Frugal Testing   

      Location                       Salary  \
0        Noida  

,Position,Company,Location,Salary,url,Job_Description
0,Machine Learning Research Engineer x 3 at SaaS...,Global Path Consulting Services LLC,"San Francisco, CA","$130,000 - $200,000 a year",https://www.simplyhired.com/job/o9ZJNHbAwRZfEC...,Machine Learning Research Engineer x 3 at SaaS...
1,ML Engineer L5 - Ads Platform Engineering (For...,Netflix,Remote,"$100,000 - $720,000 a year",https://www.simplyhired.com/job/AQfKXq6UQ156n7...,ML Engineer L5 - Ads Platform Engineering (For...
2,Sr. ML Ops Engineer,Deep 6 AI,"Pasadena, CA","$160,000 - $190,000 a year",https://www.simplyhired.com/job/W__tEkADeWsfFq...,"Sr. ML Ops Engineer\nDeep 6 AI\nPasadena, CA\n..."
3,Gen AI/ML Engineer,Ccube,"Atlanta, GA",$65 - $75 an hour,https://www.simplyhired.com/job/0RSzSQ0O3uVrZs...,"Gen AI/ML Engineer\nCcube - 3.0\nAtlanta, GA\n..."
4,"Partner 20, ML Ops Engineer",a16z,"San Francisco, CA","$225,000 - $263,000 a year",https://www.simplyhired.com/job/BwVxiQZ8gONDsz...,"Partner 20, ML Ops Engineer\na16z\nSan Francis..."
5,AI/ML Engineer,"General Technical Services, LLC","Adelphi, MD",Not specified,https://www.simplyhired.com/job/y-5yWHc0G5ve4e...,"AI/ML Engineer\nGeneral Technical Services, LL..."
6,AI-ML Engineer,Intone Networks,"Charlotte, NC",Not specified,https://www.simplyhired.com/job/J7AjxAkOrRyQVG...,AI-ML Engineer\nIntone Networks - 4.1\nCharlot...
7,Lead ML/AI Engineer,Children's National Hospital,"Silver Spring, MD","$164,071 - $273,451 a year",https://www.simplyhired.com/job/Y2Ye0o_RxOxhUe...,Lead ML/AI Engineer\nChildren's National Hospi...
8,Staff ML Engineer,Xero,"Los Angeles, CA","$150,000 - $195,000 a year",https://www.simplyhired.com/job/g9_Q-T2T1kjdbV...,"Staff ML Engineer\nXero - 4.3\nLos Angeles, CA..."
9,Senior BI Engineer,General Motors,Remote,Not specified,https://www.simplyhired.com/job/gGxorYYp4rtyQJ...,Senior BI Engineer\nGeneral Motors - 4.0\nRemo...
